# load data


In [1]:
# Paramters

In [2]:
# select test_training(1) or training(0)
test = 1

# paramters training
nbatch = 1
nepochs = 3

In [3]:
ls

/bin/bash: /home/tkeller/anaconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2D_192x192px_middleSlice/        backupFiles/
2Dpix2pix192x192_L.ipynb         ImageGenerator_2D_192x192_L.ipynb
2Dpix2pix192x192_L_Load.ipynb    metricsPlot.ipynb
2Dpix2pix192x192_L_NoMask.ipynb  oldFiles/
3Dpix2pix192x192_L.ipynb


In [4]:
cd /data-synology/anlee/

/data-synology/anlee


In [5]:
ls

/bin/bash: /home/tkeller/anaconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /bin/bash)
 COPDGene/            filenamestrain.npy*   softmasks/           'testing ccc'/
 features/            filenamesval.npy*     softmasks_summer23/   testnpy.npy*
 features_CCC/        heatmaps/             test*
 features_CCC_meas/   saliency/             test2*
 filenamestest.npy*   smoothgrad/           test3*


In [6]:
# includes&imports
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0" #for CPU ""

import shutil
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as ndi
import nilearn as nil
import tensorflow as tf

print(tf.config.list_physical_devices('GPU'))

# random seeds
np.random.seed(16)
tf.random.set_seed(16)
tf.keras.utils.set_random_seed(16)

2023-09-22 14:04:43.231684: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-22 14:04:43.781735: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


KeyboardInterrupt: 

Build an input pipeline with image paths

In [ ]:
# get image pathes for input and target images
from tqdm import tqdm

root_directory = 'COPDGene/'  # Replace with the actual path to your root directory
inputImageName = ('insp_ct_ds.nii') #('insp_ct.nii')
outputImageName = ('exp_ct_deform_ds.nii')  # Extensions of the target image files
# maskImageName = ('insp_mask_cat.nii')  # Mask image


def search_images(directory, image_list, name):
  for root, dirs, files in tqdm(os.walk(directory)):
        for file in files:
            if file.endswith(name):
                image_path = os.path.join(root, file)
                image_list.append(image_path.replace('\0', ''))  # Add the image path to the list, replace termination character

# Create an empty list to store image paths
inputImagePath = []
outputImagePath = []


# Call the search_image function with the root directory
search_images(root_directory, inputImagePath, inputImageName)
search_images(root_directory, outputImagePath, outputImageName)


print("\n")

# check if path loaded
if inputImagePath:
    print("Image input paths:")
    for path in inputImagePath[:5]:
        print(path)
else:
    print("No image files found in the directory tree.")

if outputImagePath:
    print("Image ouput paths:")
    for path in outputImagePath[:5]:
        print(path)
else:
    print("No image files found in the directory tree.")
    

In [ ]:
from sklearn.model_selection import train_test_split

# Convert image paths to lists
inputImagePath = list(inputImagePath)
outputImagePath = list(outputImagePath)

# Split the data into training and test sets list
train_input, test_input, train_output, test_output = train_test_split(
    inputImagePath, outputImagePath, test_size=0.3, random_state=42)

# Split the data into test and validation sets list
val_input, test_input, val_output, test_output = train_test_split(
    test_input, test_output, test_size=0.5, random_state=42)

print('training data: ' + str(len(train_input)))
print('validation data: ' + str(len(val_input)))
print('test data: ' + str(len(test_input)))

if train_input:
    print("Image input paths:")
    for path in train_input[:5]:
        print(path)
else:
    print("No image files found in the directory tree.")

if train_output:
    print("Image ouput paths:")
    for path in train_output[:5]:
        print(path)
else:
    print("No image files found in the directory tree.")

In [ ]:
def extract_middle_directory(file_path):
    parts = file_path.split('/')
    if len(parts) >= 3:
        return parts[1]
    return ''

In [ ]:
# print((test_input, test_output))
import csv

data_pairs = (test_input, test_output)

try:
    middle_directories = [extract_middle_directory(path) for path in test_input]
    #print(middle_directories)

    with open(f'/data-synology/tkeller/Outputs/testDataDirName_{nepochs}_{nbatch}.csv', "w") as f:
        writer = csv.writer(f)
        for input_data in middle_directories:
            writer.writerow([input_data])
    print('Data saved to', f'/data-synology/tkeller/Outputs/testDataDirName_{nepochs}_{nbatch}.csv')
    
except Exception as e:
    print("An error occurred:", str(e))

In [ ]:
print(train_input[0])
image_dataX = nib.load(train_input[0])
image = image_dataX.get_fdata()

#i,j,k = np.array(image.shape) // 2  # Assuming you want the middle slice along the first axis
# i = 95
#plt.imshow(image[i, :, :].T, cmap='Greys_r', origin='lower')
#plt.show()

In [ ]:
print(image.shape) # prints the shape of the image data

In [ ]:
image_data = np.array(image)

pixel_min = image_data.min()
pixel_max = image_data.max()

# Print the pixel range
print("Pixel range: {} - {}".format(pixel_min, pixel_max))

In [ ]:
ls

In [ ]:
# image function to load images

In [ ]:
def load_image(file_path):
    # print path to see which image is loaded
    print(file_path)
    
    # load nibable image
    image_data = nib.load(file_path).get_fdata()
    
    # find middle slice
    # i, j, k = np.array(image_data.shape) // 2

    # image = image_data[i, :, :].T

    # normalize
    pixel_min = image_data.min()
    pixel_max = image_data.max()
    image = ((image_data - pixel_min) / (pixel_max - pixel_min)) * 2 - 1

    return image

# 2D inspiratory to deformed expiratory images pix2pix from scratch

In [ ]:
os.getcwd()

In [ ]:
os.chdir("/data-synology/tkeller/")

# Implement the PatchGAN Discriminator Model


In [ ]:
os.getcwd()

In [ ]:
# example of defining a 70x70 patchgan discriminator model
#from keras.optimizers import Adam
from tensorflow.keras.optimizers.legacy import Adam
from keras.initializers import RandomNormal
from keras.models import Model
from tensorflow.keras.layers import Input
from keras.layers import Conv3D
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import BatchNormalization, GaussianNoise
#from keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import plot_model


# define the discriminator model
def define_discriminator(image_shape):
 # weight initialization
 init = RandomNormal(stddev=0.02, seed=None)
 # source image input
 in_src_image = Input(shape=image_shape)
 # target image input
 in_target_image = Input(shape=image_shape)

 # Add Gaussian noise
 # Add Gaussian Noise layer to the src image input
 # noisy_src_image = GaussianNoise(0.1)(in_src_image)  # Adjust the noise level as needed
 # Add Gaussian Noise layer to the target image input
 # noisy_target_image = GaussianNoise(0.1)(in_target_image)  # Adjust the noise level as needed

 # concatenate images channel-wise
 # merged = Concatenate()([noisy_src_image, noisy_target_image])
 merged = Concatenate()([in_src_image, in_target_image])
    
 # C64
 d = Conv3D(64, (4,4,4), strides=(2,2,2), padding='same', kernel_initializer=init)(merged)
 d = LeakyReLU(alpha=0.2)(d)
 # C128
 d = Conv3D(128, (4,4,4), strides=(2,2,2), padding='same', kernel_initializer=init)(d)
 d = BatchNormalization()(d)
 d = LeakyReLU(alpha=0.2)(d)
 # C256
 d = Conv3D(256, (4,4,4), strides=(2,2,2), padding='same', kernel_initializer=init)(d)
 d = BatchNormalization()(d)
 d = LeakyReLU(alpha=0.2)(d)
 # C512
 d = Conv3D(512, (4,4,4), strides=(2,2,2), padding='same', kernel_initializer=init)(d)
 d = BatchNormalization()(d)
 d = LeakyReLU(alpha=0.2)(d)
 # second last output layer
 d = Conv3D(512, (4,4,4), padding='same', kernel_initializer=init)(d)
 d = BatchNormalization()(d)
 d = LeakyReLU(alpha=0.2)(d)
 # patch output
 d = Conv3D(1, (4,4,4), padding='same', kernel_initializer=init)(d)
 patch_out = Activation('sigmoid')(d)
 # define model
 model = Model([in_src_image, in_target_image], patch_out)
 # compile model
 opt = Adam(learning_rate=0.0002, beta_1=0.5)
 model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5], metrics=['accuracy'])
 return model

# define image shape
image_shape = (192,192,192,1)
# create the model
model = define_discriminator(image_shape)

# summarize the model
model.summary()
# plot the model
# plot_model(model, to_file='/data-synology/tkeller/Outputs/discriminator_model_plot.png', show_shapes=True, show_layer_names=True)

# Implement the U-Net Generator Model

In [ ]:
# example of defining a u-net encoder-decoder generator model
from keras.initializers import RandomNormal
from keras.models import Model
from tensorflow.keras.layers import Input
from keras.layers import Conv3D
from keras.layers import Conv3DTranspose
from keras.layers import LeakyReLU
from keras.layers import Activation
from keras.layers import Concatenate
from keras.layers import Dropout
from keras.layers import BatchNormalization
from keras.layers import LeakyReLU
#from keras.utils.vis_utils import plot_model
from tensorflow.keras.utils import plot_model
from keras.layers import UpSampling3D

# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# add downsampling layer
	g = Conv3D(n_filters, (4,4,4), strides=(2,2,2), padding='same', kernel_initializer=init)(layer_in)
	# conditionally add batch normalization
	if batchnorm:
		g = BatchNormalization()(g, training=True)
	# leaky relu activation
	g = LeakyReLU(alpha=0.2)(g)
	return g

# define a decoder block
def decoder_block(layer_in, skip_in, n_filters, dropout=True):
	# weight initialization
	init = RandomNormal(stddev=0.02)
	# add upsampling layer
	g = Conv3DTranspose(n_filters, (4,4,4), strides=(2,2,2), padding='same', kernel_initializer=init)(layer_in)
	#g = UpSampling3D(size=(2,2,2))(layer_in)
	#g = Conv3D(n_filters,(3,3,3),strides = (1,1,1), padding='same', kernel_initializer=init)(g)
    # add batch normalization
	g = BatchNormalization()(g, training=True)
	# conditionally add dropout
	if dropout:
		g = Dropout(0.5)(g, training=True)
	# merge with skip connection
	g = Concatenate()([g, skip_in])
	# relu activation
	g = Activation('relu')(g)
	return g

# define the standalone generator model
def define_generator(image_shape=(192,192,192,1)): #image_shape=(256,256,1)
  # weight initialization
  init = RandomNormal(stddev=0.02)
  # image input
  in_image = Input(shape=image_shape)
  # encoder model: C64-C128-C256-C512-C512-C512
  e1 = define_encoder_block(in_image, 64, batchnorm=False)
  e2 = define_encoder_block(e1, 128)
  e3 = define_encoder_block(e2, 256)
  e4 = define_encoder_block(e3, 512)
  e5 = define_encoder_block(e4, 512)
  e6 = define_encoder_block(e5, 512)
  b = Conv3D(512, (3,3,3), strides=(1,1,1), padding='same', kernel_initializer=init)(e6)
  b = Activation('relu')(b)
  d1 = Conv3D(512, (3,3,3), strides=(1,1,1), padding='same', kernel_initializer=init)(b)
  d1 = Activation('relu')(d1)
  # decoder model: CD512-CD1024-CD1024-C1024-C1024-C512-C256-C128
  d2 = decoder_block(d1, e5, 512)
  d3 = decoder_block(d2, e4, 512)
  d4 = decoder_block(d3, e3, 512, dropout=False)
  d5 = decoder_block(d4, e2, 256, dropout=False)
  d6 = decoder_block(d5, e1, 128, dropout=False)
  # output
  # g = Conv2DTranspose(1, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d6)
  g = UpSampling3D(size=(2,2,2))(d6)
  g = Conv3D(1,(3,3,3),strides = (1,1,1), padding='same', kernel_initializer=init)(g)
  out_image = Activation('tanh')(g)

  # define model
  print(in_image)
  model = Model(in_image, out_image)
  return model

# define image shape
image_shape = (192,192,192,1)
# create the model
model = define_generator(image_shape)
# summarize the model
model.summary()
# plot the model
# plot_model(model, to_file='/data-synology/tkeller/generator_model_plot.png', show_shapes=True, show_layer_names=True)

# Implement Adversarial and L1 Loss


In [ ]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, image_shape):
  # make weights in the discriminator not trainable
  for layer in d_model.layers:
    if not isinstance(layer, BatchNormalization):
      layer.trainable = False
      # define the source image
      in_src = Input(shape=image_shape)
      # connect the source image to the generator input
      gen_out = g_model(in_src)
      # connect the source input and generator output to the discriminator input
      dis_out = d_model([in_src, gen_out])
      # src image as input, generated image and classification output
      model = Model(in_src, [dis_out, gen_out])
      # compile model
      opt = Adam(learning_rate=0.0002, beta_1=0.5)
      model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
      return model

In [ ]:
# define image shape
image_shape = (192,192,192,1)
# define the models
d_model = define_discriminator(image_shape)
g_model = define_generator(image_shape)
# define the composite model
gan_model = define_gan(g_model, d_model, image_shape)
# summarize the model
gan_model.summary()
# plot the model
plot_model(gan_model, to_file='/data-synology/tkeller/gan_model_plot.png', show_shapes=True, show_layer_names=True)


# Update Model Weights

In [ ]:
# from random import randint
from random import sample

# select a batch of random samples, returns images and target
def generate_real_samples(paths, n_samples, patch_shape, iterationCount):

    # random_indices = np.random.choice(range(trainA.shape[0]), size = n_samples, replace = False)
    train_input_paths, train_output_paths = paths

    # create array with random indices
    random_indices = sample(range(len(train_input_paths)), n_samples)
    print('image batch: ' + str(random_indices))

    # load random indices paths
    random_input_paths = [train_input[idx] for idx in random_indices]
    random_output_paths = [train_output[idx] for idx in random_indices]
    

    train_input_images = []
    train_output_images = []

    # for idx in random_indices:
    for input_path, output_path in zip(random_input_paths, random_output_paths):
        input_image = load_image(input_path)
        output_image = load_image(output_path)
        train_input_images.append(np.expand_dims(input_image, -1))
        train_output_images.append(np.expand_dims(output_image, -1))

    # convert to np.array
    train_dataset = (train_input_images, train_output_images)
    dataset = np.array(train_dataset)
    trainA = dataset[0]
    trainB = dataset[1]

    # generate 'real' class labels (1)
    y = np.ones((n_samples, patch_shape, patch_shape, 1))
    return [trainA, trainB], y


In [ ]:
from tensorflow.python.ops.control_flow_ops import type_spec

# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, patch_shape):
    
 # generate fake instance
 X = g_model.predict(samples)
    
 # create 'fake' class labels (0)
 y = np.zeros((len(X), patch_shape, patch_shape, 1))
    
 return X, y

In [ ]:
from matplotlib import pyplot

# create a line plot of loss for the gan and save to file
def plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist):

 # plot loss
 pyplot.subplot(2, 1, 1)
 pyplot.plot(d1_hist, label='d-real')
 pyplot.plot(d2_hist, label='d-fake')
 pyplot.plot(g_hist, label='gen')
 pyplot.legend()
    
 # plot discriminator accuracy
 pyplot.subplot(2, 1, 2)
 pyplot.plot(a1_hist, label='acc-real')
 pyplot.plot(a2_hist, label='acc-fake')
 pyplot.legend()
    
 # save plot to file
 pyplot.savefig('/data-synology/tkeller/Outputs/plot_line_plot_loss.png', dpi=1000)
 pyplot.close()

In [ ]:
import csv

# train pix2pix models
def train(d_model, g_model, gan_model, paths, n_epochs, n_batch, n_patch=12):

 # unpack tuple
 train_input_paths, train_output_paths = paths
 print('number of training inputs: ' + str(len(train_input_paths)))

 # calculate the number of batches per training epoch
 bat_per_epo = int(len(train_input_paths) / n_batch)
 print('number of batches per epoch: ' + str(bat_per_epo))

 # calculate the number of training iterations
 n_steps = int(bat_per_epo * n_epochs)
 print('number of steps: ' + str(n_steps))

 # prepare lists for storing stats each iteration
 d1_hist, d2_hist, g_hist, a1_hist, a2_hist = list(), list(), list(), list(), list()

 # manually enumerate epochs
 #for i in range(n_steps):
 for i in range(0, n_steps):
  # select a batch of real samples
  [X_realA, X_realB], y_real = generate_real_samples(paths, n_batch, n_patch, i)
  # generate a batch of fake samples
  X_fakeB, y_fake = generate_fake_samples(g_model, X_realA, n_patch)
  # update discriminator for real samples
  d_loss1, d_acc1 = d_model.train_on_batch([X_realA, X_realB], y_real)
  # update discriminator for generated samples
  d_loss2, d_acc2 = d_model.train_on_batch([X_realA, X_fakeB], y_fake)
  # update the generator
  g_loss, _, _ = gan_model.train_on_batch(X_realA, [y_real, X_realB])
  # summarize performance
  print('>%d/%d, d1[%.3f] d2[%.3f] g[%.3f], a1=%d, a2=%d' % (i+1, n_steps, d_loss1, d_loss2, g_loss, int(100*d_acc1), int(100*d_acc2)))
   # record history
  d1_hist.append(d_loss1)
  d2_hist.append(d_loss2)
  g_hist.append(g_loss)
  a1_hist.append(d_acc1)
  a2_hist.append(d_acc2)

  # number of steps to save model
  if ((i % 100) == 0 and i != 0):
    # Save the generator/discriminator model in h5
    gan_model.save(f'/data-synology/tkeller/Outputs/modelCheckp_{n_stepsX}_{nbatch}/gan_model_{n_stepsX}_{nbatch}_{i}.h5')

    # Save the generator/discriminator weights in h5
    gan_model.save_weights(f'/data-synology/tkeller/Outputs/modelCheckp_{n_stepsX}_{nbatch}/gan_model_weights_{n_stepsX}_{nbatch}_{i}.h5')

    # Save the generator/discriminator model in keras format
    gan_model.save(f'/data-synology/tkeller/Outputs/modelCheckp_{n_stepsX}_{nbatch}/gan_model_{n_stepsX}_{nbatch}_{i}.keras')

    plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist)

    rows = zip(d1_hist, d2_hist, g_hist, a1_hist, a2_hist)

    with open(f'/data-synology/tkeller/Outputs/metrics_{n_stepsX}_{nbatch}_{i}.csv', "w") as f:
        writer = csv.writer(f)
        for row in rows:
            writer.writerow(row)

    shutil.move('/data-synology/tkeller/Outputs/plot_line_plot_loss.png', f'/data-synology/tkeller/Outputs/nsteps{n_stepsX}_batch{nbatch}/plot_line_plot_loss{i}.png')
    shutil.move(f'/data-synology/tkeller/Outputs/metrics_{n_stepsX}_{nbatch}_{i}.csv', f'/data-synology/tkeller/Outputs/nsteps{n_stepsX}_batch{nbatch}/metrics_{n_stepsX}_{nbatch}_{i}.csv')

 plot_history(d1_hist, d2_hist, g_hist, a1_hist, a2_hist)


Dataset reduction for test purposes

In [ ]:
# select test or train
if test == 1:
    train_input_smaller = train_input[0::128] # every 16th item, starting from the 1th item
    train_output_smaller = train_output[0::128] # every 16th item starting from the 1th item

    train_dataset_paths = (train_input_smaller, train_output_smaller)
    
else:
    train_dataset_paths = (train_input, train_output)

# calucalte steps and so on
length = len(train_dataset_paths[0])
print('length:')
print(length)

# batch per epoch
bat_per_epoX = int(length / nbatch)
print('bat_per_epoX:')
print(bat_per_epoX)

# calculate the number of training iterations
n_stepsX = int(bat_per_epoX * nepochs)
print('n_stepsX:')
print(n_stepsX)

# create folder for "nsteps{n_stepsX}_batch{nbatch}"
#os.getcwd()
#os.chdir("/data-synology/tkeller/Outputs")
try:
    os.mkdir(f"/data-synology/tkeller/Outputs/nsteps{n_stepsX}_batch{nbatch}")
except Exception as e:
    print("An error occurred:", str(e)) 

In [ ]:
os.getcwd()

In [ ]:
os.chdir("/data-synology/anlee/")

Training loop

In [ ]:
#train_dataset_paths = (train_input, train_output)

# train model
train(d_model, g_model, gan_model, train_dataset_paths, nepochs, nbatch)

In [ ]:
ls

In [ ]:
os.getcwd()
#os.chdir("/content/gdrive/My Drive")

#os.mkdir(f"nsteps{n_stepsX}_batch{nbatch}")

In [ ]:
os.chdir("/data-synology/tkeller/Outputs")

In [ ]:
ls


In [ ]:
# save/load model

from tensorflow.keras.models import load_model

#d_model.save(f'/home/tkeller/Outputs/nsteps{n_stepsX}_batch{nbatch}/d_model_{n_stepsX}_{nbatch}.h5')
#g_model.save(f'/home/tkeller/Outputs/nsteps{n_stepsX}_batch{nbatch}/g_model_{n_stepsX}_{nbatch}.h5')
gan_model.save(f'/data-synology/tkeller/Outputs/nsteps{n_stepsX}_batch{nbatch}/gan_model_{n_stepsX}_{nbatch}.h5')
gan_model.save(f'/data-synology/tkeller/Outputs/nsteps{n_stepsX}_batch{nbatch}/gan_model_{n_stepsX}_{nbatch}.keras')
# Save the generator weights
#g_model.save_weights(f'/home/tkeller/Outputs/nsteps{n_stepsX}_batch{nbatch}/g_model_weights_{n_stepsX}_{nbatch}.h5')

gan_model.save_weights(f'/data-synology/tkeller/Outputs/nsteps{n_stepsX}_batch{nbatch}/gan_model_weights_{n_stepsX}_{nbatch}.h5')

# save model as .keras
gan_model.save(f'/data-synology/tkeller/Outputs/nsteps{n_stepsX}_batch{nbatch}/gan_model_{n_stepsX}_{nbatch}.keras')

# load model
#g_model = load_model(f'/home/tkeller/Outputs/nsteps{n_stepsX}_epochs{nepochs}/g_model_{n_stepsX}_{epochsX}.h5')

# Load the saved generator weights
#g_model.load_weights(f'/home/tkeller/Outputs/nsteps{n_stepsX}_epochs{nepochs}/g_model_weights_{n_stepsX}_{epochsX}.h5')

print("Saved models to Outputs")

Image generator


In [ ]:
def generate_images(model, test_input, tar):

    # print(test_input)
    print(test_input.shape)
    break()
    prediction = model(test_input, training=True)

    for j in range(5):
        plt.figure(figsize=(15, 15))
        display_list = [test_input[j], tar[j], prediction[j]]
        title = ['Input Image', 'Ground Truth', 'Predicted Image']

        for i in range(3):
            plt.subplot(1, 3, i+1)
            plt.title(title[i])
            # Getting the pixel values in the [0, 1] range to plot.
            plt.imshow(display_list[i], cmap='gray', origin='lower')
            plt.axis('off')
        #os.getcwd()
        #os.chdir("/home/tkeller/Outputs")
        plt.savefig(f'/data-synology/tkeller/Outputs/nsteps{n_stepsX}_batch{nbatch}/ExOutputImage_{n_stepsX}_{nbatch}_{j}.png', dpi=1000)
        plt.show()


In [ ]:
os.getcwd()
os.chdir("/data-synology/anlee/")

In [ ]:
#ls

Create datatrain for Image generator

In [ ]:
from random import sample

test_dataset = (test_input, test_output)
#print(test_dataset)

n_samples = 5
random_indices = sample(range(len(test_input)), n_samples)
print(random_indices)

print("--------")

# pick 5 random samples from validation dataset
random_input_paths_val = [val_input[idx] for idx in random_indices]
random_output_paths_val = [val_output[idx] for idx in random_indices]

val_input_images = []
val_output_images = []

#for input_path, output_path in zip(test_input, test_output):
for input_path, output_path in zip(random_input_paths_val, random_output_paths_val):
    # select random slice of 3D image: 0-191
    #i = np.random.randint(0,191)
    #print('slice i = '+ str(i))
    input_image = load_image(input_path)
    output_image = load_image(output_path)
    val_input_images.append(np.expand_dims(input_image, -1))
    val_output_images.append(np.expand_dims(output_image, -1))

val_dataset = (val_input_images, val_output_images)
#print(type(test_dataset))



In [ ]:
# create np.array
dataset = val_input_images, val_output_images
val_dataset = np.array(dataset)


Generate Fake Images

In [2]:
#for element in test_dataset[1]:

dataset = val_dataset
trainA2 = dataset[0]
trainB2 = dataset[1]

generate_images(g_model, trainA2, trainB2)

NameError: name 'val_dataset' is not defined

In [ ]:
# move files
shutil.move("/data-synology/tkeller/Outputs/plot_line_plot_loss.png", f"/data-synology/tkeller/Outputs/nsteps{n_stepsX}_batch{nbatch}/plot_line_plot_loss.png")
shutil.move("/data-synology/tkeller/Outputs/testDataDirName.csv", f"/data-synology/tkeller/Outputs/nsteps{n_stepsX}_batch{nbatch}/testDataDirName.csv")
shutil.move(f'/data-synology/tkeller/Outputs/modelCheckp_{n_stepsX}_{nbatch}', f'/data-synology/tkeller/Outputs/nsteps{n_stepsX}_batch{nbatch}/modelCheckp_{n_stepsX}_{nbatch}')


In [ ]:
#print(gan_model.summary())

In [ ]:
#weights = gan_model.get_weights()
#print(weights[0])

In [ ]:
#weights = g_model.get_weights()
#print(weights[0])